In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from pytorch_pretrained_bert import BertConfig, BertTokenizer
from model import FusionBert
from util import InputExample, InputFeatures, convert_examples_to_features

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
CONFIG_PATH = '/root/workspace/pretrained_models/uncased_L-12-H-768_A-12/bert_config.json'
# MODEL_PATH = './base_output3/pytorch_model.bin'
TOKEN_PATH = 'bert-base-uncased'
num_labels = 2
label_list = ["0", "1"]
token_length = 128

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [4]:
device = torch.device('cpu')
config = BertConfig(CONFIG_PATH)
tokenizer = BertTokenizer.from_pretrained(TOKEN_PATH)
model = FusionBert(config=config)
# model.load_state_dict(torch.load(MODEL_PATH))
model.to(device)
# model.eval()

04/02/2020 13:55:27 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


FusionBert(
  (bert_module): FusionBertModule(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLayerNorm()
                (dropout): Dropout(p=0.1)
   

In [5]:
import csv
from collections import defaultdict
datasets = []
d = defaultdict(list)
with open('../wqa_tanda/tsv_data/test.tsv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    for lidx, line in enumerate(reader):
        #if lidx == 0:
        #   continue
        d[line[0]].append(line[1:])
        
d

defaultdict(list,
            {'What do practitioners of Wicca worship ?': [['An estimated < num > Americans practice Wicca , a form of polytheistic nature worship .',
               '1'],
              ['The inch- thick chaplain handbook includes a five -page primer on Wicca , described as `` a reconstruction of the Nature worship of tribal Europe . ``',
               '1'],
              ['Wicca -- sometimes spelled Wycca -- comes from the Old English word for witch .',
               '0'],
              ['Ms . Siefferly , a senior at the high school , has practiced Wicca for several years after reading a book on the subject , Schram said .',
               '0'],
              ["That 's because Ms . Palmer is a witch , the high priestess of a group that practices Wicca at Fort Hood with the knowledge and approval of the U.S . Army .",
               '0'],
              ["Such sentiments are rooted in fear and false notions that witches worship Satan or sacrifice animals , said David 

In [6]:
def sort_and_couple(labels: np.array, scores: np.array) -> np.array:
    """Zip the `labels` with `scores` into a single list."""
    couple = list(zip(labels, scores))
    return np.array(sorted(couple, key=lambda x: x[1], reverse=True))

def meanAveragePrecision(y_true, y_pred, _threshold=0.):
    result = 0.
    pos = 0
    coupled_pair = sort_and_couple(y_true, y_pred)
    for idx, (label, score) in enumerate(coupled_pair):
        if label > _threshold:
            pos += 1.
            result += pos / (idx + 1.)
    if pos == 0:
        return 0.
    return result / pos
                    
def meanReciprocalRank(y_true, y_pred, _threshold=0.):
    coupled_pair = sort_and_couple(y_true, y_pred)
    for idx, (label, pred) in enumerate(coupled_pair):
        if label > _threshold:
            return 1. / (idx + 1)
    return 0.

In [7]:
labels = {}
datasets = {}
for k, v in d.items():
    # k is the id, v is the dataset
    dataset = []
    label = []
    for i, pair in enumerate(v):
        dataset.append([k, pair[0]])
        label.append(pair[-1])
    datasets[k] = dataset
    labels[k] = label
# labels

In [8]:
examples = []
for i, data in enumerate(dataset):
    examples.append(InputExample(i, data[0], data[1], '0'))

In [9]:
# tokenizer = BertTokenizer.from_pretrained(TOKEN_PATH)
eval_features = convert_examples_to_features(examples, label_list,
                                                 token_length, tokenizer)
eval_features

In [10]:
device = torch.device('cpu')
all_input_ids = torch.tensor(
        [f.input_ids for f in eval_features], dtype=torch.long).to(device)
all_input_mask = torch.tensor(
    [f.input_mask for f in eval_features], dtype=torch.long).to(device)
all_segment_ids = torch.tensor(
    [f.segment_ids for f in eval_features], dtype=torch.long).to(device)
all_label_ids = torch.tensor(
    [f.label_id for f in eval_features], dtype=torch.long).to(device)
x_input_ids = torch.tensor(
    [f.input_ids_x for f in eval_features], dtype=torch.long).to(device)
x_input_mask = torch.tensor(
    [f.input_mask_x for f in eval_features], dtype=torch.long).to(device)
x_segment_ids = torch.tensor(
    [f.segment_ids_x for f in eval_features], dtype=torch.long).to(device)
y_input_ids = torch.tensor(
    [f.input_ids_y for f in eval_features], dtype=torch.long).to(device)
y_input_mask = torch.tensor(
    [f.input_mask_y for f in eval_features], dtype=torch.long).to(device)
y_segment_ids = torch.tensor(
    [f.segment_ids_y for f in eval_features], dtype=torch.long).to(device)
all_input_ids

tensor([[ 101, 2129, 2146,  ...,    0,    0,    0],
        [ 101, 2129, 2146,  ...,    0,    0,    0],
        [ 101, 2129, 2146,  ...,    0,    0,    0],
        ...,
        [ 101, 2129, 2146,  ...,    0,    0,    0],
        [ 101, 2129, 2146,  ...,    0,    0,    0],
        [ 101, 2129, 2146,  ...,    0,    0,    0]])

In [11]:
logits = model(x_input_ids, x_input_mask, x_segment_ids,
                            y_input_ids, y_input_mask, y_segment_ids,
                            all_input_ids, all_segment_ids, all_input_mask)
logits

x_output tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<DivBackward1>)
simanse_embedding: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<CatBackward>)
cat_output: tensor([[-0.8356,  0.6115, -0.5656,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2846,  0.7415, -0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3172, -0.2911, -0.1849,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.3038,  0.7343, -0.7834,  ...,  0.0000,  0.0000,  0.0000],
        [-0.7590,  0.7259, -0.8178,  ...,  0.0000,  0.0000,  0.0000],
        [-0.7208,  0.4686, -0.0906,  ..., 

tensor([[-0.0547, -0.1617],
        [-0.0159, -0.1315],
        [ 0.0426, -0.1071],
        [-0.0349, -0.0668],
        [ 0.0491, -0.0845],
        [-0.0819, -0.1309],
        [-0.0011, -0.1304],
        [-0.0128, -0.1797],
        [-0.0123, -0.1992],
        [-0.0331, -0.1153],
        [-0.0097, -0.1469],
        [-0.0128, -0.1692]], grad_fn=<ThAddmmBackward>)